# OCS Stream Down Sampler 0.3

In [1]:
try:
    import ocs_academic_hub
except ImportError:
    !pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple ocs-academic-hub==0.66.0

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple
  Using cached https://files.pythonhosted.org/packages/3d/be/cebdbb03927506bd29ab2a45fdb9727ca57d00361c7099a947e08078aeb6/ocs_sample_library_preview-0.0.36rc0-py3-none-any.whl
ERROR: ocs-sample-library-preview 0.0.36rc0 has requirement python-dateutil==2.8.0, but you'll have python-dateutil 2.8.1 which is incompatible.


In [2]:
from ocs_sample_library_preview import OCSClient, SdsError, SdsStream
import configparser
import io
import json
import requests
import pandas as pd
import time 

![](https://academichub.blob.core.windows.net/images/pandas_resample_options.png)

In [26]:
stream_id = "PI_uni-pida-vm0_441" 
resample_min = '2T'
resampled_suffix = "_ds2"


# Known to work ok
batch_size = 200000
write_batch_size = 5000

# FILL OUT
namespace = "-your-value-"
tenant = "-your-value-"
version = "v1"
client_id = "-your-value-"
client_secret = "-your-value-"

In [4]:
printg=print

In [27]:
config_template = u"""
; IMPORTANT: replace these values with those provided by OSIsoft
[Configurations]
Namespace = {namespace}

[Access]
Resource = https://dat-b.osisoft.com
Tenant = {tenant}
ApiVersion = {version}

[Credentials]
ClientId = {client_id}
ClientSecret = {client_secret}
"""

In [28]:
printg(
    config_template.format(
        namespace=namespace,
        tenant=tenant,
        version=version,
        client_id=client_id,
        client_secret=client_secret,
    )
)


; IMPORTANT: replace these values with those provided by OSIsoft
[Configurations]
Namespace = -your-value-

[Access]
Resource = https://dat-b.osisoft.com
Tenant = -your-value-
ApiVersion = v1

[Credentials]
ClientId = -your-value-
ClientSecret = -your-value-



In [7]:
config = configparser.ConfigParser(allow_no_value=True)
config.read_file(
    io.StringIO(
        config_template.format(
            namespace=namespace,
            tenant=tenant,
            version=version,
            client_id=client_id,
            client_secret=client_secret,
        )
    )
)

ocs_client = OCSClient(
    config.get("Access", "ApiVersion"),
    config.get("Access", "Tenant"),
    config.get("Access", "Resource"),
    config.get("Credentials", "ClientId"),
    config.get("Credentials", "ClientSecret"),
)

namespace_id = config.get("Configurations", "Namespace")
print(f"namespace_id: '{namespace_id}'")

namespace_id: 'UC__Davis'


In [8]:
import backoff
import logging

logging.getLogger("backoff").addHandler(logging.StreamHandler())


@backoff.on_exception(backoff.expo, SdsError, max_tries=4, jitter=backoff.full_jitter)
def get_range_values(ocs_client, namespace_id, stream_id, start, batch_size):
    return ocs_client.Streams.getRangeValues(
        namespace_id,
        stream_id,
        None,
        start,
        0,  # loop * batch_size,
        batch_size,
        False,
        0,
    )

In [9]:
start_time = time.perf_counter()
loop = 0
end_of_stream = False
start_index = ocs_client.Streams.getFirstValue(namespace_id, stream_id)["Timestamp"]
while True:
    r = get_range_values(ocs_client, namespace_id, stream_id, start_index, batch_size)

    if len(r) <= 1:
        break

    if loop == 0:
        df = pd.read_json(json.dumps(r))
        df = df.set_index("Timestamp")
    else:
        tdf = pd.read_json(json.dumps(r))
        df = df.append(tdf.set_index("Timestamp"), sort=True)
    start_index = df.index[len(df) - 1].isoformat()
    print(f"[{loop}] len={len(r)}|{start_index}, ", end="")
    loop += 1
run_time = time.perf_counter() - start_time
printg(f"(done in {run_time:.2f} secs)")

[0] len=200000|2017-01-25T19:09:42.391006400+00:00, [1] len=200000|2017-02-18T11:55:18.977005+00:00, [2] len=200000|2017-03-14T09:06:28.979003900+00:00, [3] len=200000|2017-04-07T06:48:04.049011200+00:00, [4] len=200000|2017-05-01T10:48:04.800003+00:00, [5] len=200000|2017-05-27T01:45:11.116012500+00:00, [6] len=200000|2017-06-23T17:00:36.413009600+00:00, [7] len=200000|2017-07-29T09:37:35.895004200+00:00, [8] len=200000|2017-09-10T09:31:40.281005800+00:00, [9] len=200000|2017-10-08T12:54:56.513015700+00:00, [10] len=200000|2017-11-02T07:42:33.335006700+00:00, [11] len=200000|2017-12-03T16:40:19.192001300+00:00, [12] len=200000|2017-12-29T14:11:01.444000200+00:00, [13] len=200000|2018-01-23T05:22:02.875015200+00:00, [14] len=200000|2018-02-16T08:46:26.101013100+00:00, [15] len=200000|2018-03-12T21:40:47.068008400+00:00, [16] len=135072|2020-01-08T19:20:00+00:00, (done in 64.36 secs)


In [10]:
buffer = io.StringIO()
df.info(buf=buffer)
print(buffer.getvalue())
for l in buffer.getvalue().splitlines():
    pass
tokens = l.split(" ")
print(f"Read speed = {float(tokens[2][:-1])/run_time:.2f} {tokens[3]}/s")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3335072 entries, 2017-01-01 08:00:12.850006100+00:00 to 2020-01-08 19:20:00+00:00
Data columns (total 4 columns):
DigitalStateName    object
IsSubstituted       float64
SystemStateCode     float64
Value               float64
dtypes: float64(3), object(1)
memory usage: 127.2+ MB

Read speed = 1.98 MB/s


In [11]:
df.columns

Index(['DigitalStateName', 'IsSubstituted', 'SystemStateCode', 'Value'], dtype='object')

In [12]:
res_df=df['Value']

In [13]:
res_df

Timestamp
2017-01-01 08:00:12.850006100+00:00     80.995120
2017-01-01 08:00:19.706008900+00:00     83.143810
2017-01-01 08:00:29.614013600+00:00     81.324524
2017-01-01 08:00:40.389007500+00:00     83.315370
2017-01-01 08:00:49.814010600+00:00     57.817486
                                          ...    
2020-01-08 19:00:00+00:00              235.250259
2020-01-08 19:05:00+00:00              252.625076
2020-01-08 19:10:00+00:00              283.632600
2020-01-08 19:15:00+00:00              266.514800
2020-01-08 19:20:00+00:00              227.504120
Name: Value, Length: 3335072, dtype: float64

In [14]:
res_df = res_df.loc[~res_df.index.duplicated(keep="first")]

In [15]:
res_df

Timestamp
2017-01-01 08:00:12.850006100+00:00     80.995120
2017-01-01 08:00:19.706008900+00:00     83.143810
2017-01-01 08:00:29.614013600+00:00     81.324524
2017-01-01 08:00:40.389007500+00:00     83.315370
2017-01-01 08:00:49.814010600+00:00     57.817486
                                          ...    
2020-01-08 19:00:00+00:00              235.250259
2020-01-08 19:05:00+00:00              252.625076
2020-01-08 19:10:00+00:00              283.632600
2020-01-08 19:15:00+00:00              266.514800
2020-01-08 19:20:00+00:00              227.504120
Name: Value, Length: 3335056, dtype: float64

In [16]:
res_df = res_df.resample('2T').pad()

In [17]:
res_df2 = res_df.dropna()
res_df2

Timestamp
2017-01-01 08:02:00+00:00     60.653510
2017-01-01 08:04:00+00:00     55.700610
2017-01-01 08:06:00+00:00     77.653496
2017-01-01 08:08:00+00:00     58.504310
2017-01-01 08:10:00+00:00     55.965057
                                ...    
2020-01-08 19:12:00+00:00    283.632600
2020-01-08 19:14:00+00:00    283.632600
2020-01-08 19:16:00+00:00    266.514800
2020-01-08 19:18:00+00:00    266.514800
2020-01-08 19:20:00+00:00    227.504120
Name: Value, Length: 752934, dtype: float64

In [18]:
stream = ocs_client.Streams.getStream(namespace_id, stream_id)
stream

In [19]:
ds_stream = SdsStream()
ds_stream.Id = stream.Id + resampled_suffix
ds_stream.Name = stream.Name
ds_stream.Description = stream.Description
ds_stream.TypeId = stream.TypeId
ds_stream

In [20]:
ocs_client.Streams.createOrUpdateStream(namespace_id, ds_stream)

In [21]:
start_time = time.perf_counter()
batch = []
for timestamp, val in res_df2.items():
    # print(f"ts={timestamp},  val={val}")
    batch.append({"Timestamp": timestamp.isoformat(), "Value": val})
    if len(batch) % 200 == 0:
        printg(f"[{len(batch)}]", end="")
    if len(batch) >= write_batch_size:
        printg("[-write batch-]")
        ocs_client.Streams.insertValues(namespace_id, ds_stream.Id, json.dumps(batch))
        batch = []
if len(batch) > 0:
    printg(f"[write last batch len={len(batch)}]")
    ocs_client.Streams.insertValues(namespace_id, ds_stream.Id, json.dumps(batch))
run_time =  time.perf_counter() - start_time
print(f"-done in {run_time:.2f} secs -")

[200][400][600][800][1000][1200][1400][1600][1800][2000][2200][2400][2600][2800][3000][3200][3400][3600][3800][4000][4200][4400][4600][4800][5000][-write batch-]
[200][400][600][800][1000][1200][1400][1600][1800][2000][2200][2400][2600][2800][3000][3200][3400][3600][3800][4000][4200][4400][4600][4800][5000][-write batch-]
[200][400][600][800][1000][1200][1400][1600][1800][2000][2200][2400][2600][2800][3000][3200][3400][3600][3800][4000][4200][4400][4600][4800][5000][-write batch-]
[200][400][600][800][1000][1200][1400][1600][1800][2000][2200][2400][2600][2800][3000][3200][3400][3600][3800][4000][4200][4400][4600][4800][5000][-write batch-]
[200][400][600][800][1000][1200][1400][1600][1800][2000][2200][2400][2600][2800][3000][3200][3400][3600][3800][4000][4200][4400][4600][4800][5000][-write batch-]
[200][400][600][800][1000][1200][1400][1600][1800][2000][2200][2400][2600][2800][3000][3200][3400][3600][3800][4000][4200][4400][4600][4800][5000][-write batch-]
[200][400][600][800][1000][1

In [22]:
dft = pd.DataFrame(res_df2)

In [23]:
dft

,Value
Timestamp,
2017-01-01 08:02:00+00:00,60.653510
2017-01-01 08:04:00+00:00,55.700610
2017-01-01 08:06:00+00:00,77.653496
2017-01-01 08:08:00+00:00,58.504310
2017-01-01 08:10:00+00:00,55.965057
...,...
2020-01-08 19:12:00+00:00,283.632600
2020-01-08 19:14:00+00:00,283.632600
2020-01-08 19:16:00+00:00,266.514800


In [24]:
buffer = io.StringIO()
dft.info(buf=buffer)
print(buffer.getvalue())
for l in buffer.getvalue().splitlines():
    pass
tokens = l.split(" ")
print(f"Write speed = {float(tokens[2][:-1])/run_time:.2f} {tokens[3]}/s")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 752934 entries, 2017-01-01 08:02:00+00:00 to 2020-01-08 19:20:00+00:00
Data columns (total 1 columns):
Value    752934 non-null float64
dtypes: float64(1)
memory usage: 11.5 MB

Write speed = 0.64 MB/s


## Ratio downsampled / original 

In [25]:
100.0 * (len(res_df2) / (1.0 * len(df))) 

22.576244231009106